<a href="https://colab.research.google.com/github/modoukebe/depot2/blob/main/appeemapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import ee
import geemap

In [8]:
import os
import platform
from subprocess import DEVNULL, STDOUT, check_call


In [ ]:
Map = geemap.Map()
Map